In [1]:
# Duckdb is a SQL engine that allows us to execute powerful, analytics-friendly
# queries against local or remote databases and flat files.
import duckdb
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Create a database file on disk
conn = duckdb.connect('example.db')
# Enable remote access
conn.sql("INSTALL httpfs")
conn.sql("LOAD httpfs")
# This database file points to files totaling multiple GBs,
# but it's only about 300KB itself. The `ATTACH` command
# gives us access to views that sit on top of remote Parquet files.
try:
  conn.sql("ATTACH 'https://data.baseball.computer/dbt/bc_remote.db' (READ_ONLY)")
except duckdb.BinderException:
  # This command will fail if you run it more than once because it already exists,
  # in which case we don't need to do anything
  pass

conn.sql("USE bc_remote")
conn.sql("USE main_models")

In [2]:
batting_df: pd.DataFrame = conn.sql("""SELECT gl.season,
                                        gl.date,
                                        gl.doubleheader_status,
                                        gl.game_type, 
                                        eps.game_id, 
                                        ev.batter_id,
                                        ev.pitcher_id,
                                        ev.batting_side,
                                        gl.away_team_id,
                                        gl.home_team_id,
                                        eps.event_key, 
                                        eps.sequence_id, 
                                        eps.sequence_item,
                                        ev.outs,
                                        ev.base_state,
                                        ev.outs_on_play,
                                        ev.runs_on_play,
                                        ev.runs_batted_in,
                                        ev.plate_appearance_result,
                                        ev.batted_trajectory
                                    FROM event.event_pitch_sequences eps
                                    LEFT JOIN event.EVENTS EV ON eps.game_id = ev.game_id 
                                        AND eps.event_key = ev.event_key
                                    LEFT JOIN stg_gamelog GL on eps.game_id = gl.game_id 
                                    WHERE gl.season = 2023  AND gl.game_type = 'RegularSeason' and gl.away_team_id not in ('NLS', 'ALS') and gl.home_team_id not in ('NLS', 'ALS') AND gl.date < '2023-10-02' --AND eps.game_id = 'SDN202308180' --AND event_key = '654600812'
                                    GROUP BY 
                                        gl.season, 
                                        gl.date, 
                                        gl.doubleheader_status,  
                                        gl.game_type,
                                        eps.game_id, 
                                        eps.event_key, 
                                        eps.sequence_id, 
                                        eps.sequence_item, 
                                        ev.batter_id, 
                                        ev.pitcher_id, 
                                        ev.batting_side,
                                        gl.away_team_id, 
                                        gl.home_team_id, 
                                        ev.outs, 
                                        ev.base_state, 
                                        ev.outs_on_play, 
                                        ev.runs_on_play, 
                                        ev.runs_batted_in,
                                        ev.plate_appearance_result, 
                                        ev.batted_trajectory
                                    ORDER BY eps.event_key, eps.sequence_id;  
                                    """).df()


In [3]:
baserunning_df: pd.DataFrame = conn.sql(""" SELECT gl.season,
                                                gl.date,
                                                gl.game_type, 
                                                eps.game_id, 
                                                ev.batting_side,
                                                gl.away_team_id,
                                                gl.home_team_id,
                                                eps.event_key, 
                                                br.baserunner,
                                                br.runner_id,
                                                br.attempted_advance_to_base,
                                                br.baserunning_play_type,
                                                br.is_out,
                                                br.base_end,
                                                br.advanced_on_error_flag, 
                                                br.explicit_out_flag,
                                                br.run_scored_flag,
                                                br.rbi_flag,
                                                br.reached_on_event_key,
                                                br.baserunner_bit,
                                                br.is_advance_attempt,
                                                br.charge_event_id,
                                                br.explicit_charged_pitcher_id
                                            FROM event.event_pitch_sequences eps
                                            LEFT JOIN event.EVENTS EV ON eps.game_id = ev.game_id 
                                                AND eps.event_key = ev.event_key
                                            LEFT JOIN stg_gamelog GL on eps.game_id = gl.game_id 
                                            LEFT JOIN stg_event_baserunners br on eps.game_id = br.game_id
                                                AND eps.event_key = br.event_key
                                            WHERE gl.season = 2023  AND gl.game_type = 'RegularSeason' and gl.away_team_id not in ('NLS', 'ALS') and gl.home_team_id not in ('NLS', 'ALS') AND gl.date < '2023-10-02' --AND eps.game_id = 'SDN202308180' --AND event_key = '654600812'
                                            GROUP BY 
                                                gl.season, 
                                                gl.date, 
                                                gl.game_type,
                                                eps.game_id, 
                                                ev.batting_side,
                                                gl.away_team_id, 
                                                gl.home_team_id, 
                                                eps.event_key,
                                                br.runner_id,
                                                br.baserunner,
                                                br.attempted_advance_to_base,
                                                br.baserunning_play_type,
                                                br.is_out,
                                                br.base_end,
                                                br.advanced_on_error_flag, 
                                                br.explicit_out_flag,
                                                br.run_scored_flag,
                                                br.rbi_flag,
                                                br.reached_on_event_key,
                                                br.baserunner_bit,
                                                br.is_advance_attempt,
                                                br.charge_event_id,
                                                br.explicit_charged_pitcher_id 

""").df()

In [ ]:
xdf: pd.DataFrame = conn.sql("""SELECT gl.season,
                                        gl.date,
                                        gl.doubleheader_status,
                                        gl.game_type, 
                                        eps.game_id, 
                                        ev.batter_id,
                                        ev.pitcher_id,
                                        ev.batting_side,
                                        gl.away_team_id,
                                        gl.home_team_id,
                                        eps.event_key, 
                                        eps.sequence_id, 
                                        eps.sequence_item,
                                        ev.outs,
                                        ev.base_state,
                                        ev.outs_on_play,
                                        ev.runs_on_play,
                                        ev.plate_appearance_result,
                                        ev.batted_trajectory
                                    FROM stg_gamelog GL
                                    LEFT JOIN event.event_pitch_sequences eps on gl.game_id = eps.game_id 
                                    LEFT JOIN event.EVENTS EV ON gl.game_id = ev.game_id 
                                        AND eps.event_key = ev.event_key
                                    WHERE gl.season = 2023  AND gl.game_type = 'RegularSeason' and gl.away_team_id not in ('NLS', 'ALS') and gl.home_team_id not in ('NLS', 'ALS') --AND gl.date < '2023-05-01' --AND eps.game_id = 'SDN202308180' --AND event_key = '654600812'
                                    GROUP BY 
                                        gl.season, 
                                        gl.date, 
                                        gl.doubleheader_status, 
                                        gl.date, 
                                        gl.game_type,
                                        eps.game_id, 
                                        eps.event_key, 
                                        eps.sequence_id, 
                                        eps.sequence_item, 
                                        ev.batter_id, 
                                        ev.pitcher_id, 
                                        ev.batting_side,
                                        gl.away_team_id, 
                                        gl.home_team_id, 
                                        ev.outs, 
                                        ev.base_state, 
                                        ev.outs_on_play, 
                                        ev.runs_on_play, 
                                        ev.runs_batted_in,
                                        ev.plate_appearance_result, 
                                        ev.batted_trajectory
                                    ORDER BY eps.event_key, eps.sequence_id;  
                                    """).df()

In [3]:
games: pd.DataFrame = conn.sql("""SELECT * FROM stg_gamelog where
                                game_id = 'ARI202311010'""")

print(games)

┌────────┬────────────┬─────────────────────┬──────────────┬──────────────┬──────────────┬─────────────┬─────────┬────────────┬────────────────┬─────────────────┬──────────────────┬─────────────────┬──────────────────────────┬──────────────────────────┬─────────────────┬────────────────┬─────────┬───────────────┬──────────────────┬─────────────────┬─────────────────┬──────────────────┬──────────────────┬──────────────┬─────────────┐
│ season │    date    │ doubleheader_status │   game_id    │ away_team_id │ home_team_id │ time_of_day │ park_id │ attendance │ umpire_home_id │ umpire_first_id │ umpire_second_id │ umpire_third_id │ away_starting_pitcher_id │ home_starting_pitcher_id │ additional_info │ bat_first_side │ use_dh  │   game_type   │ duration_minutes │ away_line_score │ home_line_score │ away_runs_scored │ home_runs_scored │ forfeit_info │ source_type │
│ int16  │    date    │       varchar       │   varchar    │   varchar    │   varchar    │   varchar   │ varchar │   int32    

In [4]:
batting_df['event_key'] = batting_df['event_key'].astype(np.int64)
batting_df['date'] = pd.to_datetime(batting_df['date'], format='%Y-%m-%d').dt.tz_localize('UTC')

baserunning_df['event_key'] = baserunning_df['event_key'].astype(np.int64)
baserunning_df['date'] = pd.to_datetime(baserunning_df['date'], format='%Y-%m-%d').dt.tz_localize('UTC')

foul = ['Foul']
ball = ['AutomaticBall', 'Ball',  'IntentionalBall', ]
strike = ['SwingingStrike', 'CalledStrike', 'FoulBunt', 'FoulTip', 'FoulTipBunt']
other = ['InPlay', 'InPlayOnPitchout','Pitchout', 'HitByPitch','NoPitch', 'PickoffAttemptFirst', 'PickoffAttemptSecond', 'PickoffAttemptThird']

In [5]:
print(batting_df.head())

   season                      date doubleheader_status      game_type  \
0    2023 2023-04-07 00:00:00+00:00          SingleGame  RegularSeason   
1    2023 2023-04-07 00:00:00+00:00          SingleGame  RegularSeason   
2    2023 2023-04-07 00:00:00+00:00          SingleGame  RegularSeason   
3    2023 2023-04-07 00:00:00+00:00          SingleGame  RegularSeason   
4    2023 2023-04-07 00:00:00+00:00          SingleGame  RegularSeason   

        game_id batter_id pitcher_id batting_side away_team_id home_team_id  \
0  ANA202304070  sprig001   sandp002         Away          TOR          ANA   
1  ANA202304070  sprig001   sandp002         Away          TOR          ANA   
2  ANA202304070  sprig001   sandp002         Away          TOR          ANA   
3  ANA202304070  sprig001   sandp002         Away          TOR          ANA   
4  ANA202304070  sprig001   sandp002         Away          TOR          ANA   

   event_key  sequence_id   sequence_item  outs  base_state  outs_on_play  \
0  

In [6]:
batting_df2 = batting_df.copy() 

batting_df2['is_ball'] = batting_df2['sequence_item'].isin(ball)
batting_df2['is_strike'] = batting_df2['sequence_item'].isin(strike)
batting_df2['is_foul'] = batting_df2['sequence_item'].isin(foul)
batting_df2['other'] = batting_df2['sequence_item'].isin(other)

In [7]:
# Function used to create a ball-strike count for every pitch thrown in a game
countData = []

def count_function(group):
    # Initialize empty columns for counts
    group['balls_count'] = 0
    group['strikes_count'] = 0
    
    # Compute the cumulative ball count (up to a maximum of 4)
    balls_mask = group['is_ball']
    group['balls_count'] = balls_mask.cumsum().clip(upper=4)  # Clip at 4 to limit balls_count to 4
   
    # Iterate over the group to manually apply logic
    strikes = 0  # To track the strike count in the group
    for idx, row in group.iterrows():
        if row['is_strike']:
            if strikes < 3:  # Strike can increase only until 3
                strikes += 1
        elif row['is_foul']:
            if strikes < 2:  # Foul can increase the strike count until 2
                strikes += 1
        
        # Set the strike count for this row
        group.at[idx, 'strikes_count'] = strikes

    # Return the group with the updated counts
    return group[['game_id', 'event_key', 'sequence_id', 'sequence_item', 'balls_count', 'strikes_count']]

# Sample usage: Apply the count function to each group
count_df = batting_df2.groupby(['game_id', 'event_key']).apply(count_function, include_groups = True).reset_index(drop=True)

C:\Users\dillo\AppData\Local\Temp\ipykernel_40632\2642821155.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  count_df = batting_df2.groupby(['game_id', 'event_key']).apply(count_function, include_groups = True).reset_index(drop=True)


In [8]:
# Apply the transformations
batting_df2 = pd.merge(batting_df, count_df, on = ['game_id', 'event_key', 'sequence_id', 'sequence_item'], how = 'left')

# reorder columns - list all column names
cols = list(batting_df2.columns)
# find the index location of sequence index after which we want to place the count
sequence_item_idx = cols.index('sequence_item')
# reorder by 
reorder = (
    cols[:sequence_item_idx + 1] + 
    ['balls_count', 'strikes_count'] +
    [col for col in cols[sequence_item_idx + 1:] if col not in ['balls_count','strikes_count']]    
)
batting_df2 = batting_df2[reorder]

In [ ]:
batting_df2

In [9]:
# Following information from: https://docs.baseball.computer/#!/overview
# Plate Appearance Result possibilities
pa_sacrifice = ['SacrificeFly','SacrificeHit',]
pa_outs = ['FieldersChoice', 'InPlayOut',  'StrikeOut']
pa_walks = ['Walk', 'IntentionalWalk'] # Interfernce is treated as a walk: https://www.baseball-reference.com/bullpen/Error
pa_hbp = ['HitByPitch']
pa_interference = ['Interference']
pa_hits = ['Double', 'GroundRuleDouble', 'HomeRun', 'InsideTheParkHomeRun','Single', 'Triple']
pa_other = ['ReachedOnError']

# Instances where the plate appearance counts as an at-bat
pa_atbat = pa_hits.copy()
pa_atbat.extend(pa_outs)
pa_atbat.extend(pa_other)

# Instances where the 2-strike-count registers as a plate-appearance 
pa_plateappearanceTotal =  pa_atbat.copy()
pa_plateappearanceTotal.extend(pa_walks)
pa_plateappearanceTotal.extend(pa_sacrifice)
pa_plateappearanceTotal.extend(pa_hbp)
pa_plateappearanceTotal.extend(pa_interference)

# Instances where the plate appearance counts towards on-base percentage 
pa_onbaseCount = pa_hits.copy()
pa_onbaseCount.extend(pa_walks)
pa_onbaseCount.extend(pa_hbp)


# print(pa_onbaseCount)

In [10]:
#### Everything below this is new
# find all plate appearances that reached two strikes 
df2Strikes = batting_df2[batting_df2['strikes_count'] == 2]

# number of 2-strike counts
check_df2s_count = df2Strikes['event_key'].nunique()

# number of 2-strike counts with no plate appearance result in the data
check_df2s_nonresult = df2Strikes[['event_key', 'plate_appearance_result']].groupby('event_key')['plate_appearance_result'].apply(lambda x: x.isna().any()).sum()

missingpercent = check_df2s_nonresult / check_df2s_count * 100
print(missingpercent)
# ~2% missing data works for now

2.445655532075285


In [11]:
# find each of the players, and then aggregate their counting stats
# looking for plate appearances, at-bats, hits, strikeouts, walks, homeruns, doubles, triples, average, on-base percentage, slugging percentage, number of pitches seen when in a count with 2 strikes
def batting_stats(pitch_outcome):
    PA = pitch_outcome.isin(pa_plateappearanceTotal).sum()
    AB = pitch_outcome.isin(pa_atbat).sum()
    H = pitch_outcome.isin(pa_hits).sum()
    HR = pitch_outcome.isin(['HomeRun', 'InsideTheParkHomeRun']).sum()
    SO = pitch_outcome.isin(['StrikeOut']).sum()
    BB = pitch_outcome.isin(pa_walks).sum()
    HBP = pitch_outcome.isin(pa_hbp).sum()
    Singles = pitch_outcome.isin(['Single']).sum()
    Doubles = pitch_outcome.isin(['Double', 'GroundRuleDouble']).sum()
    Triples = pitch_outcome.isin(['Triple']).sum()
    SF = pitch_outcome.isin(['SacrificeFly']).sum()
    SH = pitch_outcome.isin(['SacrificeHit']).sum()
    TB = Singles + Doubles * 2 + Triples * 3 + HR * 4
    AVG = round(H / AB, 3)
    OBP = round((H + BB + HBP) / PA , 3)
    SLG = round(TB / AB, 3)
    OPS = OBP + SLG
    # Strikes2 = pitch_outcome.size()

    return pd.Series({
        'PA': PA,
        'AB': AB,
        'H': H,
        'HR': HR,
        'SO': SO,
        'BB': BB,
        'HBP': HBP,
        '2B': Doubles,
        '3B': Triples,
        'TB': TB,
        'AVG': AVG,
        'OBP': OBP,
        'SLG': SLG,
        'OPS': OPS,
        'SF': SF,
        'SH': SH
        # 'PitchCount2Strikes': Strikes2    
        })

def game_batting_stats(pitch_outcome):
    PA = pitch_outcome.isin(pa_plateappearanceTotal).sum()
    AB = pitch_outcome.isin(pa_atbat).sum()
    H = pitch_outcome.isin(pa_hits).sum()
    HR = pitch_outcome.isin(['HomeRun', 'InsideTheParkHomeRun']).sum()
    SO = pitch_outcome.isin(['StrikeOut']).sum()
    BB = pitch_outcome.isin(pa_walks).sum()
    HBP = pitch_outcome.isin(pa_hbp).sum()
    Singles = pitch_outcome.isin(['Single']).sum()
    Doubles = pitch_outcome.isin(['Double', 'GroundRuleDouble']).sum()
    Triples = pitch_outcome.isin(['Triple']).sum()
    SF = pitch_outcome.isin(['SacrificeFly']).sum()
    SH = pitch_outcome.isin(['SacrificeHit']).sum()
    TB = Singles + Doubles * 2 + Triples * 3 + HR * 4
    # Strikes2 = pitch_outcome.size()

    return pd.Series({
        'PA': PA,
        'AB': AB,
        'H': H,
        'HR': HR,
        'SO': SO,
        'BB': BB,
        'HBP': HBP,
        '2B': Doubles,
        '3B': Triples,
        'TB': TB,
        'SF': SF,
        'SH': SH
        # 'PitchCount2Strikes': Strikes2    
        })

def rbi_stats(runs_outcome):
    RBI = runs_outcome.sum()
    return pd.Series({'RBI': RBI 
                             })



In [ ]:
# baserunning calculation 
baserunning_df

In [ ]:
# need to use this df to accumulate steals and caught stealing (baserunner_type_play); runs - runner_id and run_score_flag may have to factor in addition components like base_end -  
# and RBIs - any run_scored_flag and rbi_flag both are true grouping by event key which is then joined to the game_id, event_key, batter_id df [RBIs could could from batting_df2 under runs_batted_in]

bs_df = baserunning_df.copy()

bs_df['reached_on_event_key'] = np.where((bs_df['reached_on_event_key'].isna()) & (bs_df['baserunner'] == 'Batter'), bs_df['event_key'], bs_df['reached_on_event_key'])

In [ ]:
bs_df

In [ ]:
playerInfo: pd.DataFrame = conn.sql("""SELECT DISTINCT
                                r.player_id, 
                                r.last_name, 
                                r.first_name 
                            FROM misc.roster r  
                            WHERE r.year = 2023""").df()

outcomes = batting_df2[['batter_id', 'event_key', 'plate_appearance_result']].drop_duplicates().dropna(subset = 'plate_appearance_result').groupby('batter_id')['plate_appearance_result'].apply(batting_stats).reset_index()
rbi_outcomes = batting_df2[['batter_id', 'event_key', 'runs_batted_in']].drop_duplicates().dropna(subset = 'runs_batted_in').groupby('batter_id')['runs_batted_in'].apply(rbi_stats).reset_index()

outcomes_pivot = outcomes.pivot(index='batter_id', columns='level_1', values='plate_appearance_result')
rbi_pivot = rbi_outcomes.pivot(index='batter_id', columns='level_1', values='runs_batted_in')

# Flatten the columns
outcomes_pivot.reset_index(inplace=True)
rbi_pivot.reset_index(inplace=True)

player_outcomes = pd.merge(playerInfo, outcomes_pivot, left_on = 'player_id', right_on = 'batter_id', how = 'left')
player_outcomes = pd.merge(player_outcomes, rbi_pivot, left_on = 'batter_id', right_on = 'batter_id', how = 'left')

po_reorder = (
    ['player_id', 'batter_id', 'last_name', 'first_name', 'PA', 'AB', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'SO', 'AVG', 'OBP', 'SLG', 'TB', 'HBP', 'OPS', 'SF', 'SH']  
)

player_outcomes = player_outcomes[po_reorder]

player_outcomes = player_outcomes[player_outcomes['batter_id'].notna()]

print(player_outcomes.head())

  player_id batter_id last_name first_name     PA     AB     H    2B   3B  \
2  cronc002  cronc002      Cron      C. J.  278.0  258.0  64.0  12.0  0.0   
5  padlk001  padlk001     Padlo      Kevin    8.0    8.0   1.0   1.0  0.0   
6  parik002  parik002     Paris      Kyren   46.0   40.0   4.0   0.0  0.0   
8  stefm001  stefm001  Stefanic    Michael   71.0   62.0  18.0   2.0  1.0   
9  urshg001  urshg001   Urshela        Gio  228.0  214.0  64.0   8.0  1.0   

     HR   RBI    BB    SO    AVG    OBP    SLG     TB  HBP    OPS   SF   SH  
2  12.0  37.0  17.0  65.0  0.248  0.295  0.434  112.0  1.0  0.729  2.0  0.0  
5   0.0   0.0   0.0   0.0  0.125  0.125  0.250    2.0  0.0  0.375  0.0  0.0  
6   0.0   1.0   4.0  17.0  0.100  0.196  0.100    4.0  1.0  0.296  0.0  1.0  
8   0.0   6.0   8.0   8.0  0.290  0.380  0.355   22.0  1.0  0.735  0.0  0.0  
9   2.0  24.0  10.0  36.0  0.299  0.329  0.374   80.0  1.0  0.703  3.0  0.0  


In [ ]:
game_logs = batting_df2[['batter_id', 'game_id', 'event_key', 'plate_appearance_result']].drop_duplicates().dropna(subset = 'event_key').groupby(['batter_id', 'game_id'])['plate_appearance_result'].apply(game_batting_stats).reset_index()

game_logs_pivot = game_logs.pivot(index=['batter_id', 'game_id'], columns='level_2', values='plate_appearance_result')

po_reorder = (
    ['player_id', 'batter_id', 'last_name', 'first_name', 'game_id', 'PA', 'AB', 'H', 'HR', '2B', '3B', 'SO', 'BB', 'TB', 'SF', 'SH']  
)
game_logs_pivot.reset_index(inplace=True)

game_logs_outcomes = pd.merge(playerInfo[playerInfo['player_id'] == 'bichb001'], game_logs_pivot, left_on = 'player_id', right_on = 'batter_id', how = 'left')
game_logs_outcomes = game_logs_outcomes[po_reorder]

print(game_logs_outcomes)

# game_logs_outcomes.to_csv('game_logs_check_2023.csv')

    player_id batter_id last_name first_name       game_id  PA  AB  H  HR  2B  \
0    bichb001  bichb001  Bichette         Bo  ANA202304070   4   4  2   1   1   
1    bichb001  bichb001  Bichette         Bo  ANA202304080   5   4  3   1   0   
2    bichb001  bichb001  Bichette         Bo  ANA202304090   6   5  1   0   0   
3    bichb001  bichb001  Bichette         Bo  BAL202306130   4   4  1   0   0   
4    bichb001  bichb001  Bichette         Bo  BAL202306140   4   4  2   0   1   
5    bichb001  bichb001  Bichette         Bo  BAL202306150   4   4  1   0   1   
6    bichb001  bichb001  Bichette         Bo  BAL202308220   5   5  1   0   0   
7    bichb001  bichb001  Bichette         Bo  BAL202308230   4   4  1   0   0   
8    bichb001  bichb001  Bichette         Bo  BAL202308240   4   4  1   0   0   
9    bichb001  bichb001  Bichette         Bo  BOS202305010   5   5  5   1   0   
10   bichb001  bichb001  Bichette         Bo  BOS202305020   4   2  0   0   0   
11   bichb001  bichb001  Bic

In [ ]:
# Pitcher DF: similar to accumulating stats like batter